In [1]:
# 自动计算cell的计算时间
%load_ext autotime

%matplotlib inline
%config InlineBackend.figure_format='svg' #矢量图设置，让绘图更清晰

time: 486 ms (started: 2021-08-26 12:57:40 +08:00)


In [3]:
%%bash

# 增加更新
git add *.ipynb *.md

git remote -v

git commit -m '更新 #3  Aug 26, 2021'

#git push origin master
git push

origin	git@github.com:ustchope/keras_developer_guides-.git (fetch)
origin	git@github.com:ustchope/keras_developer_guides-.git (push)
[main 9327668] 更新 #3  Aug 26, 2021
 2 files changed, 248 insertions(+), 131 deletions(-)
 create mode 100644 "\347\274\226\345\206\231\350\207\252\345\267\261\347\232\204\345\233\236\350\260\203.ipynb"


To git@github.com:ustchope/keras_developer_guides-.git
   6cd06bd..9327668  main -> main


time: 4.19 s (started: 2021-08-26 01:54:19 +08:00)


In [2]:
#设置使用的gpu
import tensorflow as tf
from tensorflow import keras

gpus = tf.config.list_physical_devices("GPU")

if gpus:
   
    gpu0 = gpus[1] #如果有多个GPU，仅使用第0个GPU
    tf.config.experimental.set_memory_growth(gpu0, True) #设置GPU显存用量按需使用
    # 或者也可以设置GPU显存为固定使用量(例如：4G)
    #tf.config.experimental.set_virtual_device_configuration(gpu0,
    #    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)]) 
    tf.config.set_visible_devices([gpu0],"GPU")

time: 5.01 s (started: 2021-08-26 12:57:44 +08:00)


# 介绍

回调是在训练、评估或推理期间自定义 Keras 模型行为的强大工具。 示例包括使用 TensorBoard 可视化训练进度和结果的 tf.keras.callbacks.TensorBoard，或在训练期间定期保存模型的 tf.keras.callbacks.ModelCheckpoint。

在本指南中，您将了解 Keras 回调是什么、它可以做什么以及如何构建自己的回调。 我们提供了一些简单的回调应用程序演示来帮助您入门。

# 设置

In [3]:
import tensorflow as tf
from tensorflow import keras

time: 373 µs (started: 2021-08-26 12:57:49 +08:00)


# Keras 回调概述
所有回调都是 keras.callbacks.Callback 类的子类，并覆盖在训练、测试和预测的各个阶段调用的一组方法。 回调对于在训练期间查看模型的内部状态和统计数据很有用。

您可以将回调列表（作为关键字参数回调）传递给以下模型方法：
* keras.Model.fit()
* keras.Model.evaluate()
* keras.Model.predict()

# 回调方法概述

## 全局方法
* `on_(train|test|predict)_begin(self,logs=None) ` 
在拟合/评估/预测开始时调用。
* `on_(train|test|predict)_end(self,logs=None)`  
在拟合/评估/预测结束时调用。

## 用于训练/测试/预测的批处理级方法
* `on_(train|test|predict)_batch_begin(self,batch,logs=None)`  
在训练/测试/预测期间处理批次之前调用。
* `on_(train|test|predict)_batch_end(self,batch,logs=None)`  
在训练/测试/预测批次结束时调用。 在此方法中，日志是一个包含指标结果的字典。

## 周期级方法（仅限训练）
* `on_epoch_begin(self, epoch, logs=None)`  
在训练期间的一个纪元开始时调用。
* `on_epoch_end(self, epoch, logs=None)`  
在训练期间的 epoch 结束时调用。

# 一个基本的例子

我们来看一个具体的例子。 首先，让我们导入 tensorflow 并定义一个简单的 Sequential Keras 模型：

In [4]:
# Define the Keras model to add callbacks to
def get_model():
    model = keras.Sequential()
    model.add(keras.layers.Dense(1, input_dim=784))
    model.compile(
        optimizer=keras.optimizers.RMSprop(learning_rate=0.1),
        loss="mean_squared_error",
        metrics=["mean_absolute_error"],
    )
    return model

time: 1.4 ms (started: 2021-08-26 13:00:15 +08:00)


然后，从 Keras 数据集 API 加载用于训练和测试的 MNIST 数据：

In [5]:
# Load example MNIST data and pre-process it
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train.reshape(-1, 784).astype("float32") / 255.0
x_test = x_test.reshape(-1, 784).astype("float32") / 255.0

# Limit the data to 1000 samples
x_train = x_train[:1000]
y_train = y_train[:1000]
x_test = x_test[:1000]
y_test = y_test[:1000]

time: 591 ms (started: 2021-08-26 13:05:02 +08:00)


现在，定义一个简单的自定义回调来记录：
* 拟合/评估/预测开始和结束时
* 当每个纪元开始和结束时
* 当每个训练批次开始和结束时
* 每个评估（测试）批次开始和结束时
* 当每个推理（预测）批次开始和结束时

In [6]:
class CustomCallback(keras.callbacks.Callback):
    def on_train_begin(self, logs=None):
        keys = list(logs.keys())
        print("Starting training; got log keys: {}".format(keys))

    def on_train_end(self, logs=None):
        keys = list(logs.keys())
        print("Stop training; got log keys: {}".format(keys))

    def on_epoch_begin(self, epoch, logs=None):
        keys = list(logs.keys())
        print("Start epoch {} of training; got log keys: {}".format(epoch, keys))

    def on_epoch_end(self, epoch, logs=None):
        keys = list(logs.keys())
        print("End epoch {} of training; got log keys: {}".format(epoch, keys))

    def on_test_begin(self, logs=None):
        keys = list(logs.keys())
        print("Start testing; got log keys: {}".format(keys))

    def on_test_end(self, logs=None):
        keys = list(logs.keys())
        print("Stop testing; got log keys: {}".format(keys))

    def on_predict_begin(self, logs=None):
        keys = list(logs.keys())
        print("Start predicting; got log keys: {}".format(keys))

    def on_predict_end(self, logs=None):
        keys = list(logs.keys())
        print("Stop predicting; got log keys: {}".format(keys))

    def on_train_batch_begin(self, batch, logs=None):
        keys = list(logs.keys())
        print("...Training: start of batch {}; got log keys: {}".format(batch, keys))

    def on_train_batch_end(self, batch, logs=None):
        keys = list(logs.keys())
        print("...Training: end of batch {}; got log keys: {}".format(batch, keys))

    def on_test_batch_begin(self, batch, logs=None):
        keys = list(logs.keys())
        print("...Evaluating: start of batch {}; got log keys: {}".format(batch, keys))

    def on_test_batch_end(self, batch, logs=None):
        keys = list(logs.keys())
        print("...Evaluating: end of batch {}; got log keys: {}".format(batch, keys))

    def on_predict_batch_begin(self, batch, logs=None):
        keys = list(logs.keys())
        print("...Predicting: start of batch {}; got log keys: {}".format(batch, keys))

    def on_predict_batch_end(self, batch, logs=None):
        keys = list(logs.keys())
        print("...Predicting: end of batch {}; got log keys: {}".format(batch, keys))

time: 1.16 ms (started: 2021-08-26 13:10:04 +08:00)


让我们试试看：

In [7]:
model = get_model()
model.fit(
    x_train,
    y_train,
    batch_size=128,
    epochs=1,
    verbose=0,
    validation_split=0.5,
    callbacks=[CustomCallback()],
)

res = model.evaluate(
    x_test, y_test, batch_size=128, verbose=0, callbacks=[CustomCallback()]
)

res = model.predict(x_test, batch_size=128, callbacks=[CustomCallback()])

Starting training; got log keys: []
Start epoch 0 of training; got log keys: []
...Training: start of batch 0; got log keys: []
...Training: end of batch 0; got log keys: ['loss', 'mean_absolute_error']
...Training: start of batch 1; got log keys: []
...Training: end of batch 1; got log keys: ['loss', 'mean_absolute_error']
...Training: start of batch 2; got log keys: []
...Training: end of batch 2; got log keys: ['loss', 'mean_absolute_error']
...Training: start of batch 3; got log keys: []
...Training: end of batch 3; got log keys: ['loss', 'mean_absolute_error']
Start testing; got log keys: []
...Evaluating: start of batch 0; got log keys: []
...Evaluating: end of batch 0; got log keys: ['loss', 'mean_absolute_error']
...Evaluating: start of batch 1; got log keys: []
...Evaluating: end of batch 1; got log keys: ['loss', 'mean_absolute_error']
...Evaluating: start of batch 2; got log keys: []
...Evaluating: end of batch 2; got log keys: ['loss', 'mean_absolute_error']
...Evaluating: 

## 日志字典的使用
`logs`字典包含损失值，以及批次或时期结束时的所有指标。 示例包括损失和平均绝对误差。

In [8]:
class LossAndErrorPrintingCallback(keras.callbacks.Callback):
    def on_train_batch_end(self, batch, logs=None):
        print(
            "Up to batch {}, the average loss is {:7.2f}.".format(batch, logs["loss"])
        )

    def on_test_batch_end(self, batch, logs=None):
        print(
            "Up to batch {}, the average loss is {:7.2f}.".format(batch, logs["loss"])
        )

    def on_epoch_end(self, epoch, logs=None):
        print(
            "The average loss for epoch {} is {:7.2f} "
            "and mean absolute error is {:7.2f}.".format(
                epoch, logs["loss"], logs["mean_absolute_error"]
            )
        )


model = get_model()
model.fit(
    x_train,
    y_train,
    batch_size=128,
    epochs=2,
    verbose=0,
    callbacks=[LossAndErrorPrintingCallback()],
)

res = model.evaluate(
    x_test,
    y_test,
    batch_size=128,
    verbose=0,
    callbacks=[LossAndErrorPrintingCallback()],
)

Up to batch 0, the average loss is   23.47.
Up to batch 1, the average loss is  446.34.
Up to batch 2, the average loss is  304.99.
Up to batch 3, the average loss is  231.12.
Up to batch 4, the average loss is  186.23.
Up to batch 5, the average loss is  156.36.
Up to batch 6, the average loss is  134.78.
Up to batch 7, the average loss is  121.21.
The average loss for epoch 0 is  121.21 and mean absolute error is    5.87.
Up to batch 0, the average loss is    4.70.
Up to batch 1, the average loss is    4.78.
Up to batch 2, the average loss is    4.83.
Up to batch 3, the average loss is    4.68.
Up to batch 4, the average loss is    4.81.
Up to batch 5, the average loss is    4.92.
Up to batch 6, the average loss is    5.23.
Up to batch 7, the average loss is    5.47.
The average loss for epoch 1 is    5.47 and mean absolute error is    1.91.
Up to batch 0, the average loss is    8.47.
Up to batch 1, the average loss is    7.42.
Up to batch 2, the average loss is    7.76.
Up to batch 

# self.model 属性的使用

除了在调用其中一个方法时接收日志信息外，回调还可以访问与当前轮次训练/评估/推理相关的模型：`self.model`。

以下是您可以在回调中使用 `self.model` 执行的一些操作：
* 设置 `self.model.stop_training = True` 立即中断训练。
* 改变优化器的超参数（可用作 `self.model.optimizer`），例如 `self.model.optimizer.learning_rate`。
* 以周期间隔保存模型。
* 在每个时期结束时在几个测试样本上记录 `model.predict()` 的输出，以用作训练期间的健全性检查。
* 在每个时期结束时提取中间特征的可视化，以监控模型随时间学习的内容。
* 等等。

让我们通过几个例子来看看它的实际效果。

# Keras 回调应用示例

* 以最小的损失提前停止
第一个示例显示了通过设置属性 `self.model.stop_training (boolean)` 创建的回调，在达到最小损失时停止训练。 或者，您可以提供一个参数耐心来指定在达到局部最小值后我们应该在停止之前等待多少个时期。

`tf.keras.callbacks.EarlyStopping` 提供了更完整和通用的实现。

In [9]:
import numpy as np


class EarlyStoppingAtMinLoss(keras.callbacks.Callback):
    """当损失达到最小值时停止训练，即损失停止减少。

   参数：
       patience：达到 min 后要等待的时代数。 在这个数量没有改进之后，训练停止。
  """

    def __init__(self, patience=0):
        super(EarlyStoppingAtMinLoss, self).__init__()
        self.patience = patience
        # best_weights to store the weights at which the minimum loss occurs.
        self.best_weights = None

    def on_train_begin(self, logs=None):
        # 当损失不再是最小值时它等待的纪元数。
        self.wait = 0
        # 训练停止的时代。
        self.stopped_epoch = 0
        # 将最好的初始化为无穷大。
        self.best = np.Inf

        
    def on_epoch_end(self, epoch, logs=None):
        current = logs.get("loss")
        if np.less(current, self.best):
            self.best = current
            self.wait = 0
            # 如果当前结果更好（更少），则记录最佳权重。
            self.best_weights = self.model.get_weights()
        else:
            self.wait += 1
            if self.wait >= self.patience:
                self.stopped_epoch = epoch
                self.model.stop_training = True
                print("Restoring model weights from the end of the best epoch.")
                self.model.set_weights(self.best_weights)

    def on_train_end(self, logs=None):
        if self.stopped_epoch > 0:
            print("Epoch %05d: early stopping" % (self.stopped_epoch + 1))


model = get_model()
model.fit(
    x_train,
    y_train,
    batch_size=64,
    steps_per_epoch=5,
    epochs=30,
    verbose=0,
    callbacks=[LossAndErrorPrintingCallback(), EarlyStoppingAtMinLoss()],
)

Up to batch 0, the average loss is   32.81.
Up to batch 1, the average loss is  439.82.
Up to batch 2, the average loss is  300.86.
Up to batch 3, the average loss is  227.67.
Up to batch 4, the average loss is  183.90.
The average loss for epoch 0 is  183.90 and mean absolute error is    8.07.
Up to batch 0, the average loss is    6.94.
Up to batch 1, the average loss is    7.09.
Up to batch 2, the average loss is    6.70.
Up to batch 3, the average loss is    6.33.
Up to batch 4, the average loss is    6.12.
The average loss for epoch 1 is    6.12 and mean absolute error is    2.00.
Up to batch 0, the average loss is    5.09.
Up to batch 1, the average loss is    5.01.
Up to batch 2, the average loss is    4.84.
Up to batch 3, the average loss is    4.84.
Up to batch 4, the average loss is    4.81.
The average loss for epoch 2 is    4.81 and mean absolute error is    1.77.
Up to batch 0, the average loss is    7.59.
Up to batch 1, the average loss is    7.68.
Up to batch 2, the avera

time: 807 ms (started: 2021-08-26 13:28:47 +08:00)


## 学习率调度
在这个例子中，我们展示了如何在训练过程中使用自定义回调来动态改变优化器的学习率。

有关更一般的实现，请参阅 `callbacks.LearningRateScheduler`。

In [10]:
class CustomLearningRateScheduler(keras.callbacks.Callback):
    """学习率调度器，根据调度设置学习率。

   参数：
       schedule：一个函数，它以纪元索引（整数，从 0 开始索引）
       和当前学习率作为输入，并返回一个新的学习率作为输出（浮点数）。
  """

    def __init__(self, schedule):
        super(CustomLearningRateScheduler, self).__init__()
        self.schedule = schedule

    def on_epoch_begin(self, epoch, logs=None):
        if not hasattr(self.model.optimizer, "lr"):
            raise ValueError('Optimizer must have a "lr" attribute.')
        # 从模型的优化器中获取当前的学习率。
        lr = float(tf.keras.backend.get_value(self.model.optimizer.learning_rate))
        # 调用 schedule 函数来获取预定的学习率。
        scheduled_lr = self.schedule(epoch, lr)
        # 在此 epoch 开始之前将值设置回优化器
        tf.keras.backend.set_value(self.model.optimizer.lr, scheduled_lr)
        print("\nEpoch %05d: Learning rate is %6.4f." % (epoch, scheduled_lr))


LR_SCHEDULE = [
    # (epoch to start, learning rate) tuples
    (3, 0.05),
    (6, 0.01),
    (9, 0.005),
    (12, 0.001),
]

def lr_schedule(epoch, lr):
    """Helper function to retrieve the scheduled learning rate based on epoch."""
    if epoch < LR_SCHEDULE[0][0] or epoch > LR_SCHEDULE[-1][0]:
        return lr
    for i in range(len(LR_SCHEDULE)):
        if epoch == LR_SCHEDULE[i][0]:
            return LR_SCHEDULE[i][1]
    return lr


model = get_model()
model.fit(
    x_train,
    y_train,
    batch_size=64,
    steps_per_epoch=5,
    epochs=15,
    verbose=0,
    callbacks=[
        LossAndErrorPrintingCallback(),
        CustomLearningRateScheduler(lr_schedule),
    ],
)


Epoch 00000: Learning rate is 0.1000.
Up to batch 0, the average loss is   27.36.
Up to batch 1, the average loss is  496.30.
Up to batch 2, the average loss is  336.32.
Up to batch 3, the average loss is  254.64.
Up to batch 4, the average loss is  205.18.
The average loss for epoch 0 is  205.18 and mean absolute error is    8.30.

Epoch 00001: Learning rate is 0.1000.
Up to batch 0, the average loss is    6.55.
Up to batch 1, the average loss is    6.30.
Up to batch 2, the average loss is    5.92.
Up to batch 3, the average loss is    6.13.
Up to batch 4, the average loss is    6.01.
The average loss for epoch 1 is    6.01 and mean absolute error is    2.03.

Epoch 00002: Learning rate is 0.1000.
Up to batch 0, the average loss is    6.92.
Up to batch 1, the average loss is    6.09.
Up to batch 2, the average loss is    6.02.
Up to batch 3, the average loss is    5.98.
Up to batch 4, the average loss is    5.62.
The average loss for epoch 2 is    5.62 and mean absolute error is    1

time: 1.41 s (started: 2021-08-26 13:42:31 +08:00)


# 内置 Keras 回调
请务必通过阅读 API 文档来查看现有的 Keras 回调。 应用程序包括记录到 CSV、保存模型、在 TensorBoard 中可视化指标等等！